# ODA data by Recipient
This tutorial shows you how to build a Pandas DataFrame containing ODA data to any number sectors and recipients, from any number of donors, over a range of years.

The tutorial has the following steps:
1. Optionally, download the raw data if you haven't yet done so
2. Import `ODAData`, the main tool used to interact with the data
3. Create an instance of `ODAData` with the specific arguments you would like to use
4. Load the indicators
5. Get the DataFrame
6. Optionally, export the DataFrame

## 1. Download the raw data

The raw data must be downloaded every time you do a fresh installation of the `oda_data` package. The raw data is not distributed with the package given its size, and to make sure users are accessing the latest data when they start working with the package.

Before getting started with analysis for the first time, you must download the raw data. This process can take a while depending on your internet connection, and on the files that you decide to download.

It is highly recommended that you specify the folder where you want to store and read the raw data. This must be done once in each notebook or script where we use the `oda_data` package.

First we will import the necessary functions. In this example, we will only download the CRS data using `download_crs(years)`, but other options include `download_dac1()`, `download_dac2a()`, and `download_multisystem()`.

To reduce file size, we set the small_version parameter to True which removes XXX. You must also set years, as below.

In [1]:
from oda_data import set_data_path

# set the path to the folder where we want to store the data
set_data_path(path="../tutorials")

## 2. Import ODAData

We can gather the data we need using the `ODAData` class. An object from this class can:
    - Get data for specific indicators
        - Optionally, filter the data for specific donors, recipients and years
        - Optionally, exchange and deflate data

You can specify the data path if the raw data has already been downloaded, or if you haven't yet specified the download path in your notebook and script.

In [2]:
from oda_data import ODAData

from oda_data.tools.groupings import recipient_groupings

## 3. Instantiating the ODAData object with the desired arguments

Next, we need to set the right arguments, which are used to create the right DataFrame.
- *years*: you must specify the `years`, as an `int`, `list` or `range`.
- *donors*: you can _optionally_ specify the `donors` you want the output to have (as `int`, or `list`) of donor codes.
- *recipients*: you can _optionally_ specify the `recipients` you want the output to have. Not all indicators need or accept recipients. If using an indicator for which recipients aren't an option, a warning will be logged to the console and the recipients ignored for that indicator.
- *currency*: you can _optionally_ specify the `currency` in which you want your data to be shown. If not specified, by default, `USD` will be used. Other options include `EUR`, `GBP` and `CAN`.
- *prices*: you can _optionally_ specify the `prices` in which you want your data to be shown. If not specified, by default, `current` will be used. The other option is `constant`. If specifying `constant` a `base_year` must be set.
- *base_year*: you must specify a `base_year` if you have set `prices = 'constant'`. If you have chosen `current` prices,
by default, `base_year` will be `None`.


Below are some example settings for this tutorial. For clarity, we will first store them in variables, but you can always pass them directly as arguments to the `ODAData` class.
You can see the available options using the .available_xxx() methods below.

In [3]:
recipient_groupings().keys()

dict_keys(['all_recipients', 'all_developing_countries_regions', 'african_countries', 'africa_regional', 'african_countries_regional', 'sahel', 'ldc_countries', 'france_priority', 'dac2a_aggregates'])

In [4]:
recipient_groupings()

{'all_recipients': {30: 'Cyprus',
  35: 'Gibraltar',
  45: 'Malta',
  55: 'Turkey',
  57: 'Kosovo',
  61: 'Slovenia',
  62: 'Croatia',
  63: 'Serbia',
  64: 'Bosnia and Herzegovina',
  65: 'Montenegro',
  66: 'North Macedonia',
  71: 'Albania',
  85: 'Ukraine',
  86: 'Belarus',
  88: 'States Ex-Yugoslavia unspecified',
  89: 'Europe, regional',
  93: 'Moldova',
  130: 'Algeria',
  133: 'Libya',
  136: 'Morocco',
  139: 'Tunisia',
  142: 'Egypt',
  189: 'North of Sahara, regional',
  218: 'South Africa',
  225: 'Angola',
  227: 'Botswana',
  228: 'Burundi',
  229: 'Cameroon',
  230: 'Cabo Verde',
  231: 'Central African Republic',
  232: 'Chad',
  233: 'Comoros',
  234: 'Congo',
  235: 'Democratic Republic of the Congo',
  236: 'Benin',
  237: 'East African Community',
  238: 'Ethiopia',
  239: 'Gabon',
  240: 'Gambia',
  241: 'Ghana',
  243: 'Guinea',
  244: 'Guinea-Bissau',
  245: 'Equatorial Guinea',
  247: "Cote d'Ivoire",
  248: 'Kenya',
  249: 'Lesotho',
  251: 'Liberia',
  252: '

In [5]:
# Select years as (for example) a range. Remember ranges are exclusive of the upper bound.
years = range(2018,2021)

# Select donors, which must be specified by their codes. To get all donors, do not use this argument.
donors = [4, 5, 12, 302]

# Select recipients, which must be specified by their codes. In this example, we have used Africa, total.
recipients = list(recipient_groupings()['african_countries_regional'])

# Select the currency. By default 'USD' is shown but we'll get the data in Euros.
currency = 'EUR'

# Select the prices. By default, 'current' is shown, but we'll get the data in constant prices.
prices = 'constant'

# Set the base year. We must set this given that we've asked for constant data.
base_year = 2021

In [6]:
ODAData().available_donors()

INFO 2023-01-06 17:06:03,798 [oda_data.py:available_donors:300] Note that not all donors may be available for all indicators


{
1: Austria,
2: Belgium,
3: Denmark,
4: France,
5: Germany,
6: Italy,
7: Netherlands,
8: Norway,
9: Portugal,
10: Sweden,
11: Switzerland,
12: United Kingdom,
18: Finland,
20: Iceland,
21: Ireland,
22: Luxembourg,
40: Greece,
50: Spain,
61: Slovenia,
68: Czech Republic,
69: Slovak Republic,
75: Hungary,
76: Poland,
301: Canada,
302: United States,
701: Japan,
742: Korea,
801: Australia,
820: New Zealand,
30: Cyprus,
45: Malta,
55: Turkey,
62: Croatia,
70: Liechtenstein,
72: Bulgaria,
77: Romania,
82: Estonia,
83: Latvia,
84: Lithuania,
87: Russia,
130: Algeria,
133: Libya,
358: Mexico,
543: Iraq,
546: Israel,
552: Kuwait,
561: Qatar,
566: Saudi Arabia,
576: United Arab Emirates,
611: Azerbaijan,
613: Kazakhstan,
732: Chinese Taipei,
764: Thailand,
765: Timor-Leste,
104: Nordic Development Fund,
807: UNEP,
811: Global Environment Facility,
812: Montreal Protocol,
901: International Bank for Reconstruction and Development,
902: Multilateral Investment Guarantee Agency,
903: Internationa

In [7]:
ODAData().available_recipients()

INFO 2023-01-06 17:06:03,818 [oda_data.py:available_recipients:306] Note that not all recipients may be available for all indicators


{
30: Cyprus,
35: Gibraltar,
45: Malta,
55: Turkey,
57: Kosovo,
61: Slovenia,
62: Croatia,
63: Serbia,
64: Bosnia and Herzegovina,
65: Montenegro,
66: North Macedonia,
71: Albania,
85: Ukraine,
86: Belarus,
88: States Ex-Yugoslavia unspecified,
89: Europe, regional,
93: Moldova,
130: Algeria,
133: Libya,
136: Morocco,
139: Tunisia,
142: Egypt,
189: North of Sahara, regional,
218: South Africa,
225: Angola,
227: Botswana,
228: Burundi,
229: Cameroon,
230: Cabo Verde,
231: Central African Republic,
232: Chad,
233: Comoros,
234: Congo,
235: Democratic Republic of the Congo,
236: Benin,
237: East African Community,
238: Ethiopia,
239: Gabon,
240: Gambia,
241: Ghana,
243: Guinea,
244: Guinea-Bissau,
245: Equatorial Guinea,
247: Cote d'Ivoire,
248: Kenya,
249: Lesotho,
251: Liberia,
252: Madagascar,
253: Malawi,
255: Mali,
256: Mauritania,
257: Mauritius,
258: Mayotte,
259: Mozambique,
260: Niger,
261: Nigeria,
265: Zimbabwe,
266: Rwanda,
268: Sao Tome and Principe,
269: Senegal,
270: Seyche

In [8]:
ODAData().available_currencies()

[
USD,
EUR,
GBP,
CAD
]

In [9]:
ODAData().available_indicators()

[
total_oda_flow_net,
total_oda_ge,
total_oda_bilateral_flow_net,
total_oda_bilateral_ge,
total_oda_multilateral_flow_net,
total_oda_multilateral_ge,
total_oda_flow_gross,
total_oda_flow_commitments,
total_oda_grants_flow,
total_oda_grants_ge,
total_oda_non_grants_flow,
total_oda_non_grants_ge,
gni,
oda_gni_flow,
od_gni_ge,
total_in_donor_students_flow,
total_in_donor_students_ge,
total_in_donor_students_ge_linked,
scholarships_flow,
scholarships_ge,
scholarships_ge_linked,
imputed_students_flow,
imputed_students_ge,
imputed_students_linked,
debt_relief_flow,
debt_relief_ge,
idrc_flow,
idrc_ge,
idrc_ge_linked,
eu_core_flow,
eu_core_ge,
eu_core_ge_linked,
total_psi_flow,
total_psi_ge,
total_psi_flow_linked,
institutional_psi_flow,
institutional_psi_ge,
institutional_psi_flow_linked,
instrument_psi_flow,
instrument_psi_ge,
instrument_psi_flow_linked,
recipient_imputed_multi_flow_net,
recipient_bilateral_flow_net,
recipient_bilateral_flow_gross,
recipient_loans_flow_gross,
recipient_loans

    ## 4. Create an instance of ODAData

In order to get the data we want, we will create an instance of the `ODAData` class using the arguments we specified above.

We will store this instance in a variable called `oda`, which we will use later to load the indicators and get the DataFrame we're after.

In [10]:
oda = ODAData(years=years,
              recipients=recipients,
              donors=donors,
              currency=currency,
              prices=prices,
              base_year=base_year,
              include_names=True)
oda.load_indicator('total_bi_multi_flow_disbursement_gross')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\MatthewPrice\\Coding\\oda_data_package\\tutorials\\crs_2018_raw.feather'

In [ ]:
df = oda.get_data()

In [11]:
import oda_data
oda_data.__version__

'0.1.8'